In [1]:
!pip install -q --upgrade pip
!pip install -q transformers accelerate peft trl bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.1 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset
import transformers
from google.colab import userdata
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GemmaTokenizer
from peft import LoraConfig

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# NF4( 4bit normalfloat)

In [7]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
     load_in_4bit=True,
     bnb_4bit_quant_type="nf4",
     bnb_4bit_compute_dtype=torch.bfloat16
 )

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
text = "Quote: I am not talented, but I'm good at not giving up "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Quote: I am not talented, but I'm good at not giving up 

I am a self-taught artist. I have been drawing since I was a child. I am a person who likes to draw and paint. I am a person who likes to paint and draw. I am a person who likes to paint and


In [10]:
text = "Quote: Imagination is more "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

output = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))


Quote: Imagination is more <strong>powerful</strong> than knowledge.

<h3>What is the meaning of the quote?</h3>

The quote is a <strong>metaphor</strong> that means that imagination is more powerful than knowledge.

<h3>What is the meaning of the quote?</h3>

The


In [11]:
os.environ["WANDB_DISABLED"] = "false"

In [13]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM"
)


In [43]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")


In [15]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [44]:
def formatting_func(example):
  quote_text = example['quote'][0] if isinstance(example['quote'], list) else example['quote']
  author_text = example['author'][0] if isinstance(example['author'], list) else example['author']
  text = f"Quote: {quote_text}\nAuthor: {author_text}"
  return text

In [45]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})

In [46]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=False,
        bf16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [47]:
trainer.train()

Step,Training Loss
1,2.657791
2,2.095176
3,2.602021
4,2.863623
5,2.448927
6,2.468192
7,2.823896
8,2.089152
9,2.903527
10,2.223797


TrainOutput(global_step=100, training_loss=1.813469830751419, metrics={'train_runtime': 268.8577, 'train_samples_per_second': 1.488, 'train_steps_per_second': 0.372, 'total_flos': 235740440186880.0, 'train_loss': 1.813469830751419})

In [48]:
text = "Quote: A Woman is like a tea bag;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: A Woman is like a tea bag; you never know how strong she is until you put her in hot water.
Author: Eleanor Roosevelt


In [49]:
text = "Quote: Outside of a dog, a book is man's"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Outside of a dog, a book is man's best friend. Inside of a dog, it's too dark to read.
Author: Grou
